# Stabilité des algorithmes

In [ ]:
import plotly.graph_objects as go
import numpy as np
from scipy import interpolate

In [ ]:
## Newton formulation 
def compute_divided_diff_coef(x, y):
    coef = y.copy()
    for i in range(1,x.size):
        coef[i:] = (coef[i:]-coef[i-1:-1])/(x[i:] - x[:-i])
    return coef

def poly_newton_interp(coef, xi, x):
    n = coef.size
    p = np.zeros(x.size)
    for i in range(n-1,0,-1):
        p = (coef[i]+p) * (x-xi[i-1])
    p = p + coef[0] 
    return p

def newton_interp(xi, yi):
    coef = compute_divided_diff_coef(xi, yi)
    return lambda x: poly_newton_interp(coef, xi, x)

## Lagrange formulation
def lagrange(k, xi, x):
    lag = np.ones(x.size)
    for i in range(xi.size):
        if (i!=k): lag *= (x-xi[i])/(xi[k]-xi[i])
    return lag

def poly_lagrange_interp(xi, yi, x):
    p = np.zeros(x.size)
    for i in range(xi.size):
        p += lagrange(i, xi, x) * yi[i]
    return p
        
def lagrange_interp(xi, yi):
    return lambda x: poly_lagrange_interp(xi, yi, x)


# Formulation barycentrique
def compute_bary_coef(xk):
    n = xk.size
    wk = np.zeros(n)
    #wk = 0. * np.copy(xk)

    for k in range(n):
        wk[k] = 1. / (np.prod( xk[k] - xk[np.arange(n)!=k] ))
        
    return wk

def poly_barycentric_interp(xk, yk, x):
    n = xk.size
    l = np.ones(x.size)
    p = np.zeros(x.size)
    
    wk = compute_bary_coef(xk)
    
    mask = np.in1d(x, xk[:])
    p[np.where(mask)] = yk[np.where(np.in1d(xk, x))]
    mask = np.invert(mask)    

    for k in range(n):
        l[mask] *= x[mask] - xk[k]
        p[mask] += (wk[k]/(x[mask] - xk[k])) * yk[k]
        
    return l*p

def barycentric_interp(xi, yi):
    return lambda x: poly_barycentric_interp(xi, yi, x)

# Formulation barycentrique modifié
def poly_barycentric_interp_modif(xk, yk, x):
    n = xk.size
    num = np.zeros(x.size)
    denom = np.zeros(x.size)
    wk = (-1)**np.arange(n+1) * np.sin(((2*np.arange(n+1)+1)*np.pi)/(2*n + 2))

    exact_x = np.zeros(x.size, dtype=bool) #pour gérer les éventuels x=x_k
    exact_values = np.array([]) #pour stocker les éventuels y_k correspondants aux x=x_k
    #np.seterr(divide='ignore', invalid='ignore') #supprime les warnings liés aux divisons par 0
    for k in range(0,n):
        diff = x - xk[k]
        num = num + wk[k] * yk[k] / diff
        denom = denom + wk[k] / diff
        find_exact = np.where(diff==0)[0]
        if find_exact.size>0:
            exact_x[find_exact] = True 
            exact_values = np.concatenate([ exact_values, np.array([yk[k]]) ])
    p = num / denom
    p[exact_x] = exact_values
    return p

def barycentric_interp_modif(xi, yi):
    return lambda x: poly_barycentric_interp_modif(xi, yi, x)

## Exemple pour la fonction $f(x) = sin(x)$

In [ ]:
def f(x):
    return np.sin(x)

### Formule de Lagrange

In [ ]:
n = 64

ximin = 0; ximax = 2*np.pi
xi = np.linspace(ximin, ximax, n+1)
yi = f(xi)

p = lagrange_interp(xi, yi)
##ps = interpolate.lagrange(xi, yi)

x = np.linspace(ximin, ximax, 500)

fig = go.Figure(layout_yaxis_range=[-1.1,1.1])
fig.add_trace(go.Scatter(x=x, y=p(x), name="p(x)"))
##fig.add_trace(go.Scatter(x=x, y=ps(x), name="p(x) scipy"))
fig.add_trace(go.Scatter(x=x, y=f(x), name="sin(x)"))
fig.add_trace(go.Scatter(x=xi, y=yi, mode='markers', name="pts d'interpolation"))
fig.show()

### Formule de Newton

In [ ]:
n = 65

ximin = 0; ximax = 2*np.pi
xi = np.linspace(ximin, ximax, n+1)
yi = f(xi)

p = newton_interp(xi, yi)
##ps = interpolate.KroghInterpolator(xi, yi)

x = np.linspace(ximin, ximax, 500)

fig = go.Figure(layout_yaxis_range=[-1.1,1.1])
fig.add_trace(go.Scatter(x=x, y=p(x), name="p(x)"))
##fig.add_trace(go.Scatter(x=x, y=ps(x), name="p(x) scipy"))
fig.add_trace(go.Scatter(x=x, y=f(x), name="sin(x)"))
fig.add_trace(go.Scatter(x=xi, y=yi, mode='markers', name="pts d'interpolation"))
fig.show()

### Formule d'interpolation barycentrique

In [ ]:
n = 60

ximin = 0; ximax = 2*np.pi
xi = np.linspace(ximin, ximax, n+1)
yi = f(xi)

p = barycentric_interp(xi, yi)
##ps = interpolate.BarycentricInterpolator(xi, yi)

x = np.linspace(ximin, ximax, 500)

fig = go.Figure(layout_yaxis_range=[-1.1,1.1])
fig.add_trace(go.Scatter(x=x, y=p(x), name="p(x)"))
##fig.add_trace(go.Scatter(x=x, y=ps(x), name="p(x) scipy"))
fig.add_trace(go.Scatter(x=x, y=f(x), name="sin(x)"))
fig.add_trace(go.Scatter(x=xi, y=yi, mode='markers', name="pts d'interpolation"))
fig.show()

### Formule d'interpolation barycentrique modifiée

In [ ]:
n = 200

ximin = 0; ximax = 2*np.pi
xi = np.linspace(ximin, ximax, n+1)
yi = f(xi)

np.seterr(divide='ignore', invalid='ignore') #supprime les warnings liés aux divisons par 0

p = barycentric_interp_modif(xi, yi)

x = np.linspace(ximin, ximax, 500)

fig = go.Figure(layout_yaxis_range=[-1.1,1.1])
fig.add_trace(go.Scatter(x=x, y=p(x), name="p(x)"))
fig.add_trace(go.Scatter(x=x, y=f(x), name="sin(x)"))
fig.add_trace(go.Scatter(x=xi, y=yi, mode='markers', name="pts d'interpolation"))
fig.show()